# Import Modules

In [5]:
# %pip install fastai
# %pip install torchsummary

In [6]:
import numpy as np
from fastai import *
from fastai.vision.all import *
from fastai.callback.all import *
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Data Paths

In [8]:
datasetDir = r'Dataset/classification'

# Importing All Models

In [9]:
# ResNet
resnet50 = models.resnet50

# MobileNet
mobilenet_v2 = models.mobilenet_v2
mobilenet_v3_large = models.mobilenet_v3_large

# DenseNet
densenet161 = models.densenet161
densenet201 = models.densenet201

# Inception
inception_v3 = models.inception_v3

# Debug

In [ ]:
learn = models.resnet50()

learn.summary()


# Common Functions

In [13]:
def load_data(path='', train='train', valid='valid', test='', size=(224,224), bs=128):
    np.random.seed(42)
    data = ImageDataLoaders.from_folder(
        path=datasetDir,
        train=train,
        valid=valid,
        item_tfms=Resize(size),
        batch_tfms=[*aug_transforms(), Normalize.from_stats(*imagenet_stats)],
        bs=bs,
        num_workers=4
    )
    return data

In [14]:
def l_learner(data, model, wd=0.1, metrics=[error_rate, accuracy, Precision(average='macro'), Recall(average='macro'), FBeta(average='macro', beta=1.0)], bnf=True, bnWd=False):
    learn = vision_learner(data, model, metrics=metrics, wd=wd, bn_final=bnf, wd_bn_bias=bnWd)
    return learn

In [15]:
def lRatefind(learn):
    learn.lr_find()
    learn.recorder.plot_lr_find()

# ResNet 50

In [16]:
data = load_data()

In [17]:
learn = l_learner(data,resnet50)

In [ ]:
lRatefind(learn)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate,accuracy,precision_score,recall_score,fbeta_score,time
0,1.146509,0.928930,0.149320,0.850680,0.854024,0.850680,0.848258,01:46
1,0.873412,0.776676,0.113946,0.886054,0.889538,0.886054,0.885367,01:42


In [ ]:
learn.unfreeze()

In [ ]:
lRatefind(learn)

In [ ]:
learn.fit_one_cycle(
    10,
    slice(1e-4, 1e-3),
    pct_start=0.8,
    cbs=[
        SaveModelCallback(monitor='accuracy', fname='resnet50'),
        ReduceLROnPlateau(monitor='accuracy', patience=10, min_delta=0.05)
    ]
)

In [ ]:
learn.export("resnet50_export.pkl")

# MobileNet

In [5]:
data = load_data()

In [ ]:
print("Train Dataset:", data.train_ds)
print("Validation Dataset:", data.valid_ds)
print("Batch Size:", data.bs)

In [ ]:
learn = l_learner(data,densenet161)

In [ ]:
lRatefind(learn)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.fit_one_cycle(2)

In [ ]:
learn.unfreeze()

In [ ]:
lRatefind(learn)

In [ ]:
learn.fit_one_cycle(
    10,
    slice(1e-4, 1e-3),
    pct_start=0.8,
    cbs=[
        SaveModelCallback(monitor='accuracy', fname='resnet152'),
        ReduceLROnPlateau(monitor='accuracy', patience=10, min_delta=0.05)
    ]
)

In [ ]:
learn.export("resnet152_export.pkl")